In [1]:
from epi_model.common_imports import *

In [2]:
from simulation_manager import simulation_manager

In [3]:
from frozendict import frozendict as fzd

# models

In [33]:
modelsets = {}

In [34]:
for name in names:
    print(f'loading {name}')
    sim_m = simulation_manager()
    sim_m.load_models(name)
    
    modelsets[name] = sim_m

loading HS1_expand300
loading HS2
loading synthetic_3


# various params

In [4]:
names = [
    'HS1_expand300',
    'HS2',
    'synthetic_3'
]

In [5]:
comparisons = {
    'VACCINATE_P': [0.05, 0.10, 0.20, 0.50],
    'R0_mean': [1, 2.5, 4],
    'INITIAL_INFECT': [5, 10, 20]
}

In [50]:
defaults = {
    'R0_mean': 2.5,
    'VACCINATE_P': 0.20,
    'INITIAL_INFECT': 20
}

In [51]:
titles = {
    "HS1_expand300": "HS-1",
    "HS2": "HS-2",
    "synthetic_3": "the synthetic network",
}
titlestr = sorted(titles)

In [52]:
strat_print = {
    'friend': "NR",
    'friendHighDegRandTop5': "NP(N=5)",
    'friendHighDegRandTop3': "NP(N=3)",
    'friendHighDegNormalErr20': "NP(ε=20)",
    'friendHighDegNormalErr10': "NP(ε=10)",
    'friendHighDeg': "NP",
    'friendHighDegChain': "NP(c)",
    'targeted': "D"
}

# some first descriptives

In [118]:
def comma_and(X):
    return ", ".join( X[:-1] ) + ", and " + X[-1]

In [133]:
desc = """
First we present the relative effectiveness of these vaccination strategies under one scenario. 
That is, with the 
    average infectiousness R0 = 2.5 (with coefficient of variance = 0.2), and 
    initially infecting 20 individuals."""
#,     and vaccinating 20% of the population.

d = dict(defaults)
d['strat'] = 'none'
d = fzd(d)

means = []
for ts in titlestr:
    m = modelsets[ts]
    inf = m.info(d)
    means.append(f"{inf['mean']:0.1f} +/- {inf['stderr']:0.1f} of the {inf['init_sus']} remaining susceptible in {titles[ts]} ({inf['mean']/inf['init_sus']:0.1%})")
means = comma_and( means )

desc += f"""
The average numbers of individuals infected in each context when we do not vaccinate at all, across 300 simulations, are {means}."""

In [134]:
pcts = []
for ts in titlestr:
    m = modelsets[ts]
    inf = m.info(d)
    pct = np.sum( np.array(inf['ninf']) >= inf['init_sus']*0.25 ) / inf['Nsims']
    pcts.append( f'{pct:0.1%} ({titles[ts]})' )
pcts = comma_and( pcts )

desc += f"""
More than 25% of the population is infected in {pcts} of simulations."""

In [135]:
display(Markdown(desc))


First we present the relative effectiveness of these vaccination strategies under one scenario. 
That is, with the 
    average infectiousness R0 = 2.5 (with coefficient of variance = 0.2), and 
    initially infecting 20 individuals.
The average numbers of individuals infected in each context when we do not vaccinate at all, across 300 simulations, are 27.5 +/- 0.7 of the 290 remaining susceptible in HS-1 (9.5%), 158.3 +/- 4.7 of the 764 remaining susceptible in HS-2 (20.7%), and 108.3 +/- 4.6 of the 980 remaining susceptible in the synthetic network (11.1%).
More than 25% of the population is infected in 0.3% (HS-1), 35.0% (HS-2), and 6.7% (the synthetic network) of simulations.

In [136]:
d2 = dict(defaults)
d2['strat'] = 'rand'
d2 = fzd(d2)

pcts = []
for ts in titlestr:
    m = modelsets[ts]
    
    inf1 = m.info(d)
    inf2 = m.info(d2)
    
    pct = 1 - inf2['P_sus_inf'] / inf1['P_sus_inf']
    var = ( inf2['stderr'] / inf2['init_sus'] ) / inf1['P_sus_inf']
    pcts.append( f'{pct:0.1%} +/- {var:0.1%} ({titles[ts]})' )
    
pcts = comma_and( pcts )

desc += f"""
Vaccinating 20% of the population randomly will decrease the average number of infections after 100 days by {pcts}."""

In [137]:
display(Markdown(desc))


First we present the relative effectiveness of these vaccination strategies under one scenario. 
That is, with the 
    average infectiousness R0 = 2.5 (with coefficient of variance = 0.2), and 
    initially infecting 20 individuals.
The average numbers of individuals infected in each context when we do not vaccinate at all, across 300 simulations, are 27.5 +/- 0.7 of the 290 remaining susceptible in HS-1 (9.5%), 158.3 +/- 4.7 of the 764 remaining susceptible in HS-2 (20.7%), and 108.3 +/- 4.6 of the 980 remaining susceptible in the synthetic network (11.1%).
More than 25% of the population is infected in 0.3% (HS-1), 35.0% (HS-2), and 6.7% (the synthetic network) of simulations.
Vaccinating 20% of the population randomly will decrease the average number of infections after 100 days by 19.8% +/- 2.2% (HS-1), 54.4% +/- 1.6% (HS-2), and 54.0% +/- 2.0% (the synthetic network).

In [138]:
d3 = dict(defaults)
d3['strat'] = 'friendHighDeg'
d3 = fzd(d3)

pcts = []
for ts in titlestr:
    m = modelsets[ts]
    
    inf = m.info(d3)

    pct = 1 - inf['P_sus_inf_rel_rand']
    var = inf['P_sus_inf_rel_rand_err']
    pcts.append( f'{pct:0.1%} +/- {var:0.1%} ({titles[ts]})' )
    
pcts = comma_and( pcts )

desc += f"""
And crucially for the purposes of this paper, nomination of most popular contacts (NP) does significantly better than random vaccination. 
Vaccinating 20% of the population in this way decreases the average number of infections after 100 days by a *further* {pcts}, relative to random vaccination."""

In [139]:
display(Markdown(desc))


First we present the relative effectiveness of these vaccination strategies under one scenario. 
That is, with the 
    average infectiousness R0 = 2.5 (with coefficient of variance = 0.2), and 
    initially infecting 20 individuals.
The average numbers of individuals infected in each context when we do not vaccinate at all, across 300 simulations, are 27.5 +/- 0.7 of the 290 remaining susceptible in HS-1 (9.5%), 158.3 +/- 4.7 of the 764 remaining susceptible in HS-2 (20.7%), and 108.3 +/- 4.6 of the 980 remaining susceptible in the synthetic network (11.1%).
More than 25% of the population is infected in 0.3% (HS-1), 35.0% (HS-2), and 6.7% (the synthetic network) of simulations.
Vaccinating 20% of the population randomly will decrease the average number of infections after 100 days by 19.8% +/- 2.2% (HS-1), 54.4% +/- 1.6% (HS-2), and 54.0% +/- 2.0% (the synthetic network).
And crucially for the purposes of this paper, nomination of most popular contacts (NP) does significantly better than random vaccination. 
Vaccinating 20% of the population in this way decreases the average number of infections after 100 days by a *further* 20.4% +/- 2.3% (HS-1), 22.1% +/- 2.8% (HS-2), and 43.6% +/- 2.4% (the synthetic network), relative to random vaccination.